# Titanic

## Librerie

In [277]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sl.__version__)

2.2.0
1.0.4
1.18.4
0.22.2.post1


## Download dei Dataset

In [0]:
url_train = 'https://raw.githubusercontent.com/rirolli/Titanic/master/train.csv'
url_test = 'https://raw.githubusercontent.com/rirolli/Titanic/master/test.csv'

titanic_load_train = pd.read_csv(url_train)
titanic_load_test = pd.read_csv(url_test)

In [279]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Ottimizzazione del Dataset

In [0]:
# rimozione della colonna dei nomi delle persone in quanto non è necessaria
# ai fini dell'apprendimento della rete
titanic_load_train.pop('Name')
titanic_load_test.pop('Name')

# Codifica di tutti i dati dei dataset titanic_load_train e titanic_load_test
for elem_train in titanic_load_train:
  titanic_load_train[elem_train] = (pd.Categorical(titanic_load_train[elem_train])).codes

for elem_test in titanic_load_test:
  titanic_load_test[elem_test] = (pd.Categorical(titanic_load_test[elem_test])).codes


In [281]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,2,1,28,1,0,523,18,-1,2
1,1,1,0,0,51,1,0,596,207,81,0
2,2,1,2,0,34,0,0,669,41,-1,2
3,3,1,0,0,47,1,0,49,189,55,2
4,4,0,2,1,47,0,0,472,43,-1,2


In [282]:
# Stampa delle prima 5 righe del dataset di test
titanic_load_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,1,44,0,0,152,24,-1,1
1,1,2,0,60,1,0,221,5,-1,2
2,2,1,1,74,0,0,73,41,-1,1
3,3,2,1,34,0,0,147,34,-1,2
4,4,2,0,27,1,1,138,46,-1,2


### Ripartizione del Dataset

In [0]:
# Dividiamo il train set in due parti così da poter addestrare la rete:
# y che contiene solo la colonna 'Survived' che consiste nella soluzione al problema
# X che contiene tutte le altre colonne.
y = titanic_load_train.Survived
X = titanic_load_train.drop(labels=['Survived'], axis=1)

# I Dati vengono, a loro volta, divisi in altre due parti: una per
# l'addestramento e una per la valutazione della rete.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)

# I dati vengono poi memorizzati dentro un Dataset di Tensorflow per sfruttarli
# per l'addestramento. Questa stessa operazione viene poi fatta anche per il
# test set e per l'evaluetion set.
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))).shuffle(len(X_train)).batch(1)
val_dataset = (tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))).shuffle(len(X_val)).batch(1)
test_dataset = (tf.data.Dataset.from_tensor_slices(titanic_load_test.values)).batch(1)

## La rete neurale

In [0]:
num_inputs = X_train.shape[1]

In [285]:
model = keras.Sequential([
  tf.keras.layers.Dense(32, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_62 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 33        
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


## Addestramento della rete

In [286]:
# Addestramento della rete neurale
model.fit(train_dataset, epochs=300)

Epoch 1/300
712/712 [==============================] - 1s 1ms/step - loss: 4.4466 - accuracy: 0.5913
Epoch 2/300
712/712 [==============================] - 1s 1ms/step - loss: 1.9370 - accuracy: 0.6278
Epoch 3/300
712/712 [==============================] - 1s 1ms/step - loss: 1.6559 - accuracy: 0.6250
Epoch 4/300
712/712 [==============================] - 1s 1ms/step - loss: 1.5281 - accuracy: 0.6461
Epoch 5/300
712/712 [==============================] - 1s 1ms/step - loss: 1.2962 - accuracy: 0.6489
Epoch 6/300
712/712 [==============================] - 1s 1ms/step - loss: 1.1997 - accuracy: 0.6376
Epoch 7/300
712/712 [==============================] - 1s 1ms/step - loss: 1.0091 - accuracy: 0.6517
Epoch 8/300
712/712 [==============================] - 1s 1ms/step - loss: 1.0490 - accuracy: 0.6461
Epoch 9/300
712/712 [==============================] - 1s 1ms/step - loss: 0.8500 - accuracy: 0.6685
Epoch 10/300
712/712 [==============================] - 1s 1ms/step - loss: 0.8106 - accura

## Valutazione della rete

In [287]:
# Valutazione del modello
val_loss, val_acc = model.evaluate(val_dataset)
print("\nTest accuracy: {:.2f} ({:.2%})".format(val_acc, val_acc))

179/179 [==============================] - 0s 889us/step - loss: 0.8875 - accuracy: 0.7486

Test accuracy: 0.75 (74.86%)


## Previsione

In [0]:
predictions = model.predict(test_dataset)

In [291]:
print("passegerID \t Predicted survival\n")
for pid, prediction in zip(titanic_load_test['PassengerId'], predictions):
  prediction = tf.sigmoid(prediction).numpy()
  print("{} \t\t Predicted survival: {:.2%}".format(pid, prediction[0]))

passegerID 	 Predicted survival

0 		 Predicted survival: 0.00%
1 		 Predicted survival: 0.00%
2 		 Predicted survival: 0.06%
3 		 Predicted survival: 0.00%
4 		 Predicted survival: 99.36%
5 		 Predicted survival: 11.03%
6 		 Predicted survival: 0.00%
7 		 Predicted survival: 0.22%
8 		 Predicted survival: 27.26%
9 		 Predicted survival: 0.03%
10 		 Predicted survival: 2.73%
11 		 Predicted survival: 0.01%
12 		 Predicted survival: 100.00%
13 		 Predicted survival: 100.00%
14 		 Predicted survival: 0.00%
15 		 Predicted survival: 38.02%
16 		 Predicted survival: 0.39%
17 		 Predicted survival: 0.00%
18 		 Predicted survival: 0.30%
19 		 Predicted survival: 0.00%
20 		 Predicted survival: 1.41%
21 		 Predicted survival: 4.00%
22 		 Predicted survival: 99.99%
23 		 Predicted survival: 0.64%
24 		 Predicted survival: 94.29%
25 		 Predicted survival: 0.00%
26 		 Predicted survival: 100.00%
27 		 Predicted survival: 0.00%
28 		 Predicted survival: 0.00%
29 		 Predicted survival: 0.65%
30 		